In [4]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2

In [7]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print('GPU trouvé: {}'.format(tf.test.gpu_device_name()))
else:
    print("Aucun GPU trouvé")

Aucun GPU trouvé


2023-05-13 16:17:45.553315: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-13 16:17:45.553415: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
images = [] # liste pour stocker toutes les images
labels = [] # liste pour stocker les labels de chaque image

dataset_directory = "images"

for breed in os.listdir(dataset_directory):
    breed_directory = os.path.join(dataset_directory, breed)
    if os.path.isdir(breed_directory):
        for image_filename in os.listdir(breed_directory):
            img = cv2.imread(os.path.join(breed_directory, image_filename))
            if img is not None:
                images.append(img)
                labels.append(breed)

KeyboardInterrupt: 

In [ ]:
#Redimensionnement des images

image_size = (224, 224) # Vous pouvez choisir une autre taille

images = [cv2.resize(img, image_size) for img in images]

In [ ]:
#Normalisation des images

for i in range(len(images)):
    images[i] = images[i] / 255.0

In [ ]:
#Conversion des labels en encodage one-hot

from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
from sklearn.model_selection import train_test_split

# Convertir les listes en numpy arrays
images = np.array(images)
labels = np.array(labels)

# Séparation du jeu de données
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

: 

: 

In [9]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

def process_images_from_directory(directory, batch_size):
    images = []
    labels = []
    count = 0

    for breed in os.listdir(directory):
        breed_directory = os.path.join(directory, breed)
        if os.path.isdir(breed_directory):
            for image_filename in os.listdir(breed_directory):
                img = cv2.imread(os.path.join(breed_directory, image_filename))
                if img is not None:
                    img = cv2.resize(img, (224, 224)) # redimensionner l'image
                    img = img / 255.0 # normaliser l'image
                    images.append(img)
                    labels.append(breed)
                    count += 1

                    # Si le nombre d'images atteint la taille du batch, traitez ce batch et commencez un nouveau batch
                    if count >= batch_size:
                        yield np.array(images), np.array(labels)
                        images = []
                        labels = []
                        count = 0

    # Traitez le dernier batch qui pourrait être inférieur à la taille du batch
    if count > 0:
        yield np.array(images), np.array(labels)

# Utilisation de la fonction
directory = "images"
batch_size = 100 # Choisissez une taille de batch qui convient à votre machine

for images, labels in process_images_from_directory(directory, batch_size):
    # Utilisez ici les images et les labels pour former votre modèle
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [10]:
#Construction du modèle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

num_classes = len(np.unique(y_train)) # nombre de races de chiens

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3))) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) # La dernière couche a un nombre de neurones égal au nombre de classes

In [11]:
#Compilation du modèle

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
#Entraînement du modèle

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10


/home/lolonois/.local/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(32, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 1s 404ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10


/home/lolonois/.local/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2/2 [==============================] - 1s 298ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 1s 300ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - 1s 312ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
2/2 [==============================] - 1s 298ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
2/2 [==============================] - 1s 297ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
2/2 [==============================] - 1s 310ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 8/10
2/2 [==============================] - 1s 292ms/step - loss: 0.0

In [13]:
#Evaluation du modèle

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

1/1 [==============================] - 0s 40ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Loss: 0.0
Accuracy: 0.0


In [14]:
#Sauvegarde du modèle

model.save('models/model.h5')